# 04: Uncertainty Quantification

**Level:** Advanced | **Time:** ~20 min

## Overview
Complete UQ workflow: parameter uncertainty → predictive uncertainty → validation.

In [ ]:
import sys
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from src.config import CONFIG
from src.hierarchical import hierarchical_case2
from src.solvers import solve_biofilm

np.random.seed(42)
output_dir = Path('figures')
output_dir.mkdir(exist_ok=True)
print('✓ Setup complete')

## 1. Run Calibration and Extract Posterior

In [ ]:
print('Running calibration...')
results = hierarchical_case2(CONFIG)
samples_M1 = results.tmcmc_M1.samples[-1]
print(f'✓ Got {len(samples_M1)} posterior samples')

## 2. Parameter Uncertainty (Posterior Statistics)

In [ ]:
# Compute statistics
mean = np.mean(samples_M1, axis=0)
std = np.std(samples_M1, axis=0)
ci_lower = np.percentile(samples_M1, 2.5, axis=0)
ci_upper = np.percentile(samples_M1, 97.5, axis=0)

print('Parameter Uncertainty (95% Credible Intervals):')
print('='*60)
for i in range(5):
    print(f'θ[{i}]: {mean[i]:.4f} ± {std[i]:.4f}  [{ci_lower[i]:.4f}, {ci_upper[i]:.4f}]')
print('='*60)

## 3. Forward Uncertainty Propagation

In [ ]:
# Sample posterior and generate predictions
n_pred = 100
idx = np.random.choice(len(samples_M1), n_pred, replace=False)
samples_subset = samples_M1[idx]

print(f'Generating {n_pred} forward predictions...')
predictions = []
for theta in samples_subset:
    pred = solve_biofilm(theta, CONFIG['M1'])
    predictions.append(pred)
predictions = np.array(predictions)
print('✓ Predictions complete')

## 4. Visualize Predictive Uncertainty

In [ ]:
mean_pred = np.mean(predictions, axis=0)
std_pred = np.std(predictions, axis=0)
lower_95 = np.percentile(predictions, 2.5, axis=0)
upper_95 = np.percentile(predictions, 97.5, axis=0)

t = np.linspace(0, CONFIG['M1']['maxtimestep'] * CONFIG['M1']['dt'], len(mean_pred))

fig, ax = plt.subplots(figsize=(10, 6))
ax.fill_between(t, lower_95[:, 0], upper_95[:, 0], alpha=0.3, label='95% CI')
ax.plot(t, mean_pred[:, 0], 'b-', linewidth=2, label='Mean')
ax.set_xlabel('Time', fontsize=12)
ax.set_ylabel('Species 1 Volume Fraction', fontsize=12)
ax.set_title('Predictive Uncertainty', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.savefig(output_dir / '04_uncertainty_bands.png', dpi=150)
plt.show()

## 5. Posterior Predictive Checks

Validate that observations fall within predicted uncertainty.

In [ ]:
# Compute standardized residuals
sigma_obs = CONFIG.get('sigma_obs', 0.005)
residuals = predictions - mean_pred[np.newaxis, ...]
standardized = residuals / sigma_obs

# Coverage check
within_2sigma = np.abs(standardized) < 2
coverage = np.mean(within_2sigma)

print(f'Coverage within 2σ: {coverage*100:.1f}% (expect ~95%)')

# Plot residual histogram
fig, ax = plt.subplots(figsize=(8, 5))
ax.hist(standardized.flatten(), bins=50, density=True, alpha=0.7)
x = np.linspace(-4, 4, 100)
ax.plot(x, np.exp(-x**2/2)/np.sqrt(2*np.pi), 'r-', linewidth=2, label='N(0,1)')
ax.set_xlabel('Standardized Residuals', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.set_title('Residual Analysis', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.savefig(output_dir / '04_residuals.png', dpi=150)
plt.show()

## Summary

### UQ Workflow
1. Parameter uncertainty → Posterior samples
2. Predictive uncertainty → Forward propagation
3. Validation → Posterior predictive checks

### Key Metrics
- **Credible intervals**: 95% CI for parameters
- **Prediction bands**: Uncertainty in forecasts
- **Coverage**: Do observations fall within predictions?

### Applications
- Risk assessment
- Decision making under uncertainty
- Model validation

### Next: `05_advanced_visualization.ipynb`